# Validation

In this notebook we validate the `IOclassifier` against a reference design with a DDR3 pinout (i.e. a couple of bidir SSTL pins, a couple of output SSTL pins, disabled pins, ...)

In [1]:
import csv
import enum
import os
from collections import namedtuple

import numpy as np

from JicBitstream import JicBitstream, JicBitstreamZip
from IOclassifier import IOclassifier, IOSTD_REL_TO_PU, IOclassifierOut

%matplotlib notebook

In [2]:
pin_list = open("../../resources/pin_list_5SGSMD5K1F40C1_8A.txt", "r").readlines()
pin_list = [pin.strip() for pin in pin_list]

In [3]:
filename = "../../bitstreams/validation/ddr1.txt"

VALIDATION_NAME = {}
VALIDATION_DIR = {}
VALIDATAION_IO_STD = {}


with open(filename, "r", encoding="iso-8859-1") as f:
    reader = csv.reader(f, delimiter=":", skipinitialspace=True)
    [next(reader) for _ in range(85)]
    hdr = [el.strip() for el in next(reader)]
    next(reader)
    pin_name_idx = hdr.index("Pin Name/Usage")
    loc_idx = hdr.index("Location")
    dir_idx = hdr.index("Dir.")
    io_std_idx = hdr.index("I/O Standard")
    for arr in reader:
        loc = arr[loc_idx].strip()
        if loc in pin_list:
            VALIDATION_NAME[loc] = arr[pin_name_idx].strip()
            VALIDATION_DIR[loc] = arr[dir_idx].strip()
            VALIDATAION_IO_STD[loc] = arr[io_std_idx].strip()

In [4]:
from IPython.display import display, Markdown

jic_val1 = JicBitstream("../../bitstreams/validation/ddr1.jic")
iocls = IOclassifier()

y_ = iocls.classify(jic_val1, pin_list)


@enum.unique
class TermColor(enum.Enum):
    RED = "\x1b[31m"
    GREEN = "\x1b[32m"
    BLUE = "\x1b[34m"
    MAGENTA = "\x1b[35m"
    CYAN = "\x1b[36m"
    RESET = "\x1b[0m"


def get_color(pin: IOclassifierOut):

    if pin.inp and pin.out and pin.diff:
        return TermColor.RED.value
    elif pin.inp and pin.out:
        return TermColor.BLUE.value
    elif pin.out:
        return TermColor.GREEN.value
    elif pin.inp:
        return TermColor.MAGENTA.value
    elif not pin.inp and not pin.out:
        return TermColor.CYAN.value
    return TermColor.RESET.value

for pin, pin_l in zip(pin_list, y_):
    col = get_color(pin_l)
    print(
        col,
        pin,
        ",",
        VALIDATION_NAME[pin],
        ",",
        VALIDATION_DIR[pin],
        ",",
        VALIDATAION_IO_STD[pin],
        TermColor.RESET.value,
    )
    print(" " * 8, pin_l)
    print()

 R32 , memory_mem_odt[0] , output , SSTL-15 Class I 
         IOclassifierOut(inp=False, out=True, pu=False, io_std='Scls1', term='no term', diff=False)

 P32 , memory_mem_cas_n[0] , output , SSTL-15 Class I 
         IOclassifierOut(inp=False, out=True, pu=False, io_std='Scls1', term='no term', diff=False)

 U31 , memory_mem_cke[0] , output , SSTL-15 Class I 
         IOclassifierOut(inp=False, out=True, pu=False, io_std='Scls1', term='no term', diff=False)

 T31 , memory_mem_a[3] , output , SSTL-15 Class I 
         IOclassifierOut(inp=False, out=True, pu=False, io_std='Scls1', term='no term', diff=False)

 N32 , memory_mem_a[1] , output , SSTL-15 Class I 
         IOclassifierOut(inp=False, out=True, pu=False, io_std='Scls1', term='no term', diff=False)

 M32 , memory_mem_a[2] , output , SSTL-15 Class I 
         IOclassifierOut(inp=False, out=True, pu=False, io_std='Scls1', term='no term', diff=False)

 N33 , memory_mem_we_n[0] , output , SSTL-15 Class I 
         IOclassifierOut(i